# CNN classifier on word embeddings 

In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd
import spacy
import numpy as np

In [14]:
SEED = 42

In [15]:
df = pd.read_csv('raw_data/fulltrain.csv', header=None, index_col = False)
X_train = df.iloc[:, 1] 
y_train = df.iloc[:, 0]

### Preprocessing

In [16]:
spacy_model = spacy.load("en_core_web_lg")

def preprocess(sentence):
    '''
    Preprocessing strategies:
    1) Tokenization
    2) Punctuation removal
    3) Lowercase
    '''
    tokens = spacy_model(sentence)
    ls_sentence = [token.text.lower() for token in tokens if not token.is_punct]
    return ls_sentence

In [17]:
# To be used by features for feature extraction:
# X_train = X_train.apply(preprocess)
# X_train = X_train.apply(lambda sentence: ' '.join(sentence))

# Save pre-processed data
# preprocess_filename = "strip_punct_lower"
# compression_opts = dict(method='zip', archive_name=f'{preprocess_filename}.csv')
# X_train.to_csv(f'{preprocess_filename}.zip', index=False, compression=compression_opts)

In [18]:
# quickload pre-processed data
preprocess_filename = "strip_punct_lower"
# replace 'raw_data/*.csv' with .csv file containing preprocessed data
X_train = pd.read_csv(f'{preprocess_filename}.csv', index_col=False).iloc[:, 0]

### Train-Validation Split

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)

### Feature extraction: Word embeddings

In [22]:
def get_upper_quartile_sentence_length(sentences):
    UPPER_QUARTILE = 75
    lengths = []
    for sentence in sentences:
        lengths.append(len(sentence.split()))
    return int(np.percentile(lengths, UPPER_QUARTILE))

word_embedding_length = get_upper_quartile_sentence_length(X_train)

721


In [21]:
spacy_model = spacy.load("en_core_web_lg")

def get_word_embedding(sentence, word_embedding_length):
    SPACY_WORD_VECTOR_DIMENSION = 300
    res_matrix = np.zeros((word_embedding_length, SPACY_WORD_VECTOR_DIMENSION))
    row = 0
    for word in sentence:
        res_matrix[row] = spacy_model(word).vector
        row += 1
    return res_matrix

X_train_embeddings = X_train.apply(lambda sentence: get_word_embedding(sentence, word_embedding_length))

IndexError: index 721 is out of bounds for axis 0 with size 721